# Simulando Filas e Epidemias
## Modelagem e Avaliação de Desempenho (MAD)
## Grupo

- Ana Carolina Ferreira de Figueiredo
- Andrew Faria
- Daniel Arruda Ponte
- Paulo Yamagishi

# Imports e Constantes

In [ ]:
import numpy as np
import heapq
import scipy
import pandas as pd
import math
import random

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
CHEGADA = 0
SERVICO = 1

# Simulador (eventos discretos)

## Implementação

### Simulador (lista de eventos)

In [ ]:
def simulador(taxa_chegada, taxa_servico, servico_limite):
   tempo_atual = 0
   clientes = 0
   numero_servicos = 0
   eventos_log = []

   rng = np.random.default_rng()

   lista_de_eventos = [(rng.exponential(1/taxa_chegada), CHEGADA)]

   while numero_servicos < servico_limite:  # Tempo em que a simulação ocorre
    tempo_evento, tipo_evento = heapq.heappop(lista_de_eventos) # Pego o evento de maior prioridade (próximo evento de acordo com o tempo)

    if tipo_evento == CHEGADA:
      tempo_atual = tempo_evento # Mudo o tempo atual para o tempo do próximo evento
      clientes += 1
      eventos_log.append((tempo_atual, CHEGADA, clientes))
      tempo_chegada = rng.exponential(1/taxa_chegada) + tempo_atual # Calculando o tempo da próxima chegada
      heapq.heappush(lista_de_eventos, (tempo_chegada, CHEGADA))  # Coloco na fila de prioridade o próximo evento de chegada

      if clientes == 1: # Agendar o próximo serviço quando resta apenas uma pessoa na fila
        tempo_servico = rng.exponential(1/taxa_servico) + tempo_atual
        heapq.heappush(lista_de_eventos, (tempo_servico, SERVICO))

    elif tipo_evento == SERVICO:
      tempo_atual = tempo_evento # Mudo o tempo atual para o tempo do próximo evento
      clientes -= 1
      eventos_log.append((tempo_atual, SERVICO, clientes))
      numero_servicos += 1

      if clientes > 0: # Agendar o próximo atendimento
        tempo_servico = rng.exponential(1/taxa_servico) + tempo_atual
        heapq.heappush(lista_de_eventos, (tempo_servico, SERVICO))

   return eventos_log

### Métricas de Interesse

In [ ]:
def calcular_tempo_espera(eventos_log):
  chegadas = []
  tempo_espera = []

  for tempo_evento, tipo_evento, _ in eventos_log:
    if tipo_evento == CHEGADA:
      heapq.heappush(chegadas, tempo_evento)
    else:
      tempo_espera.append(tempo_evento - heapq.heappop(chegadas))

  return tempo_espera

In [ ]:
def calcular_tempo_medio(eventos_log):
  return np.mean(calcular_tempo_espera(eventos_log))

In [ ]:
def calcular_media_clientes(eventos_log):
  soma_clientes = 0

  for i in range(len(eventos_log) - 1):
    soma_clientes += (eventos_log[i + 1][0] - eventos_log[i][0]) * eventos_log[i][2]

  return soma_clientes / eventos_log[-1][0]

In [ ]:
def calcular_periodo_ocupado(eventos_log, C = 1, U = 0, limite_finito = 10000):
  tempo_inicial = -1
  periodo_ocupado = []

  finitos = 0
  infinitos = 0

  for tempo_evento, tipo_evento, clientes in eventos_log:
      if tipo_evento == CHEGADA and clientes == C and tempo_inicial == -1:
        tempo_inicial = tempo_evento

        if C == U:
          periodo_ocupado.append(0)
          tempo_inicial = -1

      elif tipo_evento == CHEGADA and tempo_inicial >= 0 and clientes == limite_finito and finitos == 0:
        infinitos = 1

      elif tipo_evento == SERVICO and clientes == U and tempo_inicial >= 0:
        tempo_final = tempo_evento
        periodo_ocupado.append(tempo_final - tempo_inicial)
        tempo_inicial = -1

        if infinitos == 0:
          finitos = 1

  return False if len(periodo_ocupado) == 0 else periodo_ocupado, finitos, infinitos

In [ ]:
def calcular_num_visitas(eventos_log):
  num_clientes = [clientes for tempo_evento, tipo_evento, clientes in eventos_log]

  pi = np.zeros(max(num_clientes) + 1)

  for i in range(len(eventos_log) - 1):
    pi[eventos_log[i][2]] += eventos_log[i + 1][0] - eventos_log[i][0]

  return pi

### Diversas simulações

In [ ]:
def n_simulacoes(n, taxa_chegada, taxa_servico, servico_limite, simulador, C = 1, U = 0, limite_finito = 10000):
  tempo_espera = []
  clientes_sistema = []
  periodo_ocupado = []
  clientes_restantes = []

  periodos_finitos = []
  periodos_infinitos = []

  num_visitas = []

  for i in range(n):
    lista_eventos = simulador(taxa_chegada, taxa_servico, servico_limite)

    # Tempo de espera
    tempo_espera.append(calcular_tempo_medio(lista_eventos))

    # Clientes no sistema
    clientes_sistema.append(calcular_media_clientes(lista_eventos))

    # Período ocupado
    periodo_ocupado, finitos, infinitos = calcular_periodo_ocupado(lista_eventos, C, U, limite_finito)

    if periodo_ocupado != False:
      periodo_ocupado.append(np.mean(periodo_ocupado))

    periodos_finitos.append(finitos)
    periodos_infinitos.append(infinitos)

    # Clientes restantes no sistema
    clientes_restantes.append(lista_eventos[-1][2])

    # Número de visitas a cada estado
    num_visitas.append(calcular_num_visitas(lista_eventos))

  max_estados = max([len(i) for i in num_visitas])

  num_visitas_matriz = np.zeros((len(num_visitas), max_estados))

  for i in range(len(num_visitas)):
    for j in range(len(num_visitas[i])):
      num_visitas_matriz[i][j] = num_visitas[i][j]

  return dict(
    tempo_espera = tempo_espera,
    clientes_sistema = clientes_sistema,
    periodo_ocupado = periodo_ocupado,
    clientes_restantes = clientes_restantes,
    periodos_finitos = periodos_finitos,
    periodos_infinitos = periodos_infinitos,
    num_visitas = num_visitas_matriz
  )

### Intervalo de Confiança

In [ ]:
def intervalo_confianca(lista):
  conf_int = scipy.stats.norm.interval(0.95, loc=np.mean(lista), scale=np.std(lista))

  return conf_int

### CDF

In [ ]:
def fila_cdf(tempo_espera, clientes_sistema):
  tempo_espera = np.sort(tempo_espera)
  cdf_tempo_espera = np.arange(len(tempo_espera)) / float(len(tempo_espera))

  clientes_sistema = np.sort(clientes_sistema)
  cdf_clientes_sistema = np.arange(len(clientes_sistema)) / float(len(clientes_sistema))

  return tempo_espera, cdf_tempo_espera, clientes_sistema, cdf_clientes_sistema

# M/M/1 Básica e Variantes

## $ρ = 0.5, λ = 1, µ = 2$

### Tempo de espera e Clientes no sistema

#### Simulação

In [ ]:
parametros = n_simulacoes(300, 1, 2, 5000, simulador)

tempo_espera = parametros['tempo_espera']
clientes_sistema = parametros['clientes_sistema']

In [ ]:
#@title Estimativas e intervalo de confiança

print('Tempo de espera')
print(f'Intervalo de confiança: {intervalo_confianca(tempo_espera)}')
print(f'Média: {np.mean(tempo_espera)}\n')

print('Clientes no sistema')
print(f'Intervalo de confiança: {intervalo_confianca(clientes_sistema)}')
print(f'Média: {np.mean(clientes_sistema)}\n')

fig = make_subplots(rows=1, cols=2, subplot_titles=('Tempo de espera', 'Clientes no sistema'))
fig.add_trace(go.Histogram(x=tempo_espera, name='Tempo de espera'), row=1, col=1)
fig.add_trace(go.Histogram(x=clientes_sistema, name='Clientes no sistema'), row=1, col=2)

fig.update_layout(height=600, width=1200)
fig.show()

Tempo de espera
Intervalo de confiança: (0.9097415762401682, 1.084696399399684)
Média: 0.9972189878199261

Clientes no sistema
Intervalo de confiança: (0.8940165188245778, 1.0992837557759956)
Média: 0.9966501373002866



In [ ]:
#@title CDF

tempo_espera, cdf_tempo_espera, clientes_sistema, cdf_clientes_sistema = fila_cdf(tempo_espera, clientes_sistema)

fig = make_subplots(rows=1, cols=2, subplot_titles=('Tempo de espera', 'Clientes no sistema'))

fig.add_trace(go.Scatter(x=tempo_espera, y=cdf_tempo_espera, name="Tempo de espera", line_shape='linear'), row=1, col=1)
fig.add_trace(go.Scatter(x=clientes_sistema, y=cdf_clientes_sistema, name="Clientes no sistema", line_shape='linear'), row=1, col=2)

fig.update_layout(height=600, width=1200)
fig.show()

## $ρ = 0.5, λ = 2, µ = 4$

### Tempo de espera e Clientes no sistema

#### Simulação

In [ ]:
parametros = n_simulacoes(300, 2, 4, 5000, simulador)

tempo_espera = parametros['tempo_espera']
clientes_sistema = parametros['clientes_sistema']

In [ ]:
#@title Estimativas e intervalo de confiança

print('Tempo de espera')
print(f'Intervalo de confiança: {intervalo_confianca(tempo_espera)}')
print(f'Média: {np.mean(tempo_espera)}\n')

print('Clientes no sistema')
print(f'Intervalo de confiança: {intervalo_confianca(clientes_sistema)}')
print(f'Média: {np.mean(clientes_sistema)}\n')

fig = make_subplots(rows=1, cols=2, subplot_titles=('Tempo de espera', 'Clientes no sistema'))
fig.add_trace(go.Histogram(x=tempo_espera, name='Tempo de espera'), row=1, col=1)
fig.add_trace(go.Histogram(x=clientes_sistema, name='Clientes no sistema'), row=1, col=2)

fig.update_layout(height=600, width=1200)
fig.show()

Tempo de espera
Intervalo de confiança: (0.45672021510183575, 0.5422521660460334)
Média: 0.4994861905739346

Clientes no sistema
Intervalo de confiança: (0.8955290115045609, 1.0996197286391385)
Média: 0.9975743700718497



In [ ]:
#@title CDF

tempo_espera, cdf_tempo_espera, clientes_sistema, cdf_clientes_sistema = fila_cdf(tempo_espera, clientes_sistema)

fig = make_subplots(rows=1, cols=2, subplot_titles=('Tempo de espera', 'Clientes no sistema'))

fig.add_trace(go.Scatter(x=tempo_espera, y=cdf_tempo_espera, name="Tempo de espera", line_shape='linear'), row=1, col=1)
fig.add_trace(go.Scatter(x=clientes_sistema, y=cdf_clientes_sistema, name="Clientes no sistema", line_shape='linear'), row=1, col=2)

fig.update_layout(height=600, width=1200)
fig.show()

## $ρ = 2, λ = 4, µ = 2$

#### Tempo de espera e Clientes no sistema

#### Simulação

In [ ]:
parametros = n_simulacoes(300, 4, 2, 5000, simulador)

tempo_espera = parametros['tempo_espera']
clientes_sistema = parametros['clientes_sistema']

In [ ]:
#@title Estimativas e intervalo de confiança

print('Tempo de espera')
print(f'Intervalo de confiança: {intervalo_confianca(tempo_espera)}')
print(f'Média: {np.mean(tempo_espera)}\n')

print('Clientes no sistema')
print(f'Intervalo de confiança: {intervalo_confianca(clientes_sistema)}')
print(f'Média: {np.mean(clientes_sistema)}\n')

fig = make_subplots(rows=1, cols=2, subplot_titles=('Tempo de espera', 'Clientes no sistema'))
fig.add_trace(go.Histogram(x=tempo_espera, name='Tempo de espera'), row=1, col=1)
fig.add_trace(go.Histogram(x=clientes_sistema, name='Clientes no sistema'), row=1, col=2)

fig.update_layout(height=600, width=1200)
fig.show()

Tempo de espera
Intervalo de confiança: (581.6649006147945, 671.3798133329601)
Média: 626.5223569738773

Clientes no sistema
Intervalo de confiança: (2320.4385015438493, 2689.801445746345)
Média: 2505.119973645097



In [ ]:
#@title CDF

tempo_espera, cdf_tempo_espera, clientes_sistema, cdf_clientes_sistema = fila_cdf(tempo_espera, clientes_sistema)

fig = make_subplots(rows=1, cols=2, subplot_titles=('Tempo de espera', 'Clientes no sistema'))

fig.add_trace(go.Scatter(x=tempo_espera, y=cdf_tempo_espera, name="Tempo de espera", line_shape='linear'), row=1, col=1)
fig.add_trace(go.Scatter(x=clientes_sistema, y=cdf_clientes_sistema, name="Clientes no sistema", line_shape='linear'), row=1, col=2)

fig.update_layout(height=600, width=1200)
fig.show()

# Simulação para Verificar Resultados Analíticos

### Fila escolhida

- $λ = 1$
- $μ = 10$
- $ρ = 0.1$

## Proporção de visitas

### Simulação

In [ ]:
parametros = n_simulacoes(300, 1, 10, 10000, simulador, limite_finito=100)

num_visitas = parametros['num_visitas']

In [ ]:
proporcao_visitas = np.zeros(np.shape(num_visitas))

for i in range(len(num_visitas)):
  proporcao_visitas[i] = num_visitas[i] / sum(num_visitas[i])

media_visitas = np.zeros(len(proporcao_visitas[0]))
erro_visitas = np.zeros(len(proporcao_visitas[0]))

intervalo_confianca_visitas = []

for j in range(len(media_visitas)):
  media_visitas[j] = np.mean(proporcao_visitas[:, j])
  erro_visitas[j] = abs(intervalo_confianca(proporcao_visitas[:, j])[1] - media_visitas[j])
  intervalo_confianca_visitas.append(intervalo_confianca(proporcao_visitas[:, j]))

In [ ]:
#@title Tabela

df = pd.DataFrame({
    'Proporção da média de visitas': media_visitas,
    'Intervalo de confiança -': [abs(i[0]) for i in intervalo_confianca_visitas],
    'Intervalo de confiança +': [abs(i[1]) for i in intervalo_confianca_visitas],
})

df

,Proporção da média de visitas,Intervalo de confiança -,Intervalo de confiança +
0,8.999495e-01,8.970783e-01,9.028208e-01
1,9.001339e-02,8.759359e-02,9.243318e-02
2,9.032675e-03,8.171777e-03,9.893574e-03
3,9.069226e-04,6.236685e-04,1.190177e-03
4,8.853136e-05,3.407153e-06,1.736556e-04
5,8.262622e-06,1.699971e-05,3.352495e-05
6,6.932584e-07,6.532139e-06,7.918656e-06
7,2.620440e-09,8.618876e-08,9.142964e-08


In [ ]:
#@title Gráfico

fig = go.Figure(data=go.Scatter(
      x=list(range(len(media_visitas))),
      y=media_visitas,
      mode='markers',
      error_y=dict(
          type='data',
          array=erro_visitas
      )
  ))

fig.show()

### Cadeia de Markov (Método da potência)

In [ ]:
#@title Gráfico

N = len(media_visitas)
lambda_ = 1
mu = 10

P = np.zeros((N, N))

P_chegada = lambda_ / (mu + lambda_)
P_saida = mu / (mu + lambda_)

P[0][0] = P_saida
P[0][1] = P_chegada

for i in range(1, N - 1):
  P[i][i - 1] = P_saida
  P[i][i + 1] = P_chegada

P_n = np.linalg.matrix_power(P, 100)

fig = px.scatter(x=range(len(P_n[0])), y=(P_n[0] / sum(P_n[0])))
fig.show()

# Fração de períodos ocupados que terminam


## $ρ = 0.5, λ = 1, µ = 2$

In [ ]:
parametros = n_simulacoes(300, 1, 2, 5000, simulador, limite_finito=2)

periodos_finitos = parametros['periodos_finitos']
periodos_infinitos = parametros['periodos_infinitos']

In [ ]:
arvores_finitas_fracoes = []

for i in range(len(periodos_finitos)):
  arvores_finitas_fracoes.append(periodos_finitos[i] / (periodos_finitos[i] + periodos_infinitos[i]))

print(f'\nIntervalo de confiança: {intervalo_confianca(arvores_finitas_fracoes)}')
print(f'Média: {np.mean(arvores_finitas_fracoes)}\n')


Intervalo de confiança: (-0.24587839910352238, 1.5925450657701892)
Média: 0.6733333333333333



## $ρ = 0.5, λ = 2, µ = 4$

In [ ]:
parametros = n_simulacoes(300, 2, 4, 5000, simulador, limite_finito=2)

periodos_finitos = parametros['periodos_finitos']
periodos_infinitos = parametros['periodos_infinitos']

In [ ]:
arvores_finitas_fracoes = []

for i in range(len(periodos_finitos)):
  arvores_finitas_fracoes.append(periodos_finitos[i] / (periodos_finitos[i] + periodos_infinitos[i]))

print(f'\nIntervalo de confiança: {intervalo_confianca(arvores_finitas_fracoes)}')
print(f'Média: {np.mean(arvores_finitas_fracoes)}\n')


Intervalo de confiança: (-0.2572692162331186, 1.590602549566452)
Média: 0.6666666666666666



## $ρ = 2, λ = 4, µ = 2$

In [ ]:
parametros = n_simulacoes(300, 4, 2, 5000, simulador, limite_finito=2)

periodos_finitos = parametros['periodos_finitos']
periodos_infinitos = parametros['periodos_infinitos']

In [ ]:
arvores_finitas_fracoes = []

for i in range(len(periodos_finitos)):
  arvores_finitas_fracoes.append(periodos_finitos[i] / (periodos_finitos[i] + periodos_infinitos[i]))

print(f'\nIntervalo de confiança: {intervalo_confianca(arvores_finitas_fracoes)}')
print(f'Média: {np.mean(arvores_finitas_fracoes)}\n')


Intervalo de confiança: (-0.593437361601932, 1.2401040282685984)
Média: 0.3233333333333333



# Inferência e MCMC

## Implementação

### Inferência do $ρ$ (MCMC)

In [ ]:
# Função para calcular a verossimilhança de ρ dado um traço de dados
def likelihood(trace, rho, K=1):
  L = 1

  if 0 < rho and rho < 1:
    for v in trace:
      L *= ((1 - rho) * rho**v)/K

    return L
  else:
    return 0

# Função para executar o algoritmo MCMC e obter a distribuição de ρ
def mcmc_rho(trace, num_iteracoes, burn_in, proposal_range, K=1):
  limite_inferior = proposal_range[0]
  limite_superior = proposal_range[1]

  rho_amostras = []
  rho_atual = np.random.uniform(0, 1)

  for i in range(num_iteracoes + burn_in):
    rho_proposto = rho_atual + np.random.uniform(limite_inferior, limite_superior)
    rho_proposto = max(0, min(1, rho_proposto))

    likelihood_atual = likelihood(trace, rho_atual, K)

    if likelihood_atual == 0:
      rho_atual = rho_proposto
    else:
      aceitacao_prob = likelihood(trace, rho_proposto, K) / likelihood_atual

      if np.random.uniform(0, 1) < aceitacao_prob:
        rho_atual = rho_proposto

    if i >= burn_in:
      rho_amostras.append(rho_atual)

  return rho_amostras

### Inferência do trace (MCMC)

In [ ]:
def clientes_restantes_prob(rho, n):
    return (1 - rho) * rho**n

def mcmc_trace(rho, m):
    trace = []
    estado_atual = 0

    for _ in range(m):
        estado_proposto = max(0, estado_atual + random.choice([-1, 1]))  # Proposta de mudança de estado
        aceitacao_prob = clientes_restantes_prob(rho, estado_proposto) / clientes_restantes_prob(rho, estado_atual)

        if np.random.uniform(0, 1) < aceitacao_prob:
            estado_atual = estado_proposto

        trace.append(estado_atual)

    return trace

## Inferência de $ρ$ via amostras do simulador

### $m = 100, ρ = 0.5$

- $λ = 1$
- $μ = 2$

In [ ]:
parametros = n_simulacoes(100, 1, 2, 5000, simulador)

clientes_restantes = parametros['clientes_restantes']

In [ ]:
#@title Inferência de ρ

trace = clientes_restantes
num_iteracoes = 12000
burn_in = 2000

rho_amostras = mcmc_rho(trace, num_iteracoes, burn_in, (-0.1, 0.1))

fig = go.Figure(go.Histogram(x=rho_amostras, name='Distribuição de ρ estimada (PDF)'))

fig.update_layout(
  height=600,
  width=800,
  title_text='Distribuição de ρ estimada (PDF)',
  xaxis_title_text='ρ',
  yaxis_title_text='PDF'
)

fig.show()

print(f'\nIntervalo de confiança: {intervalo_confianca(rho_amostras)}')
print(f'Média: {np.mean(rho_amostras)}\n')

### $m = 300, ρ = 0.5$

- $λ = 1$
- $μ = 2$

In [ ]:
parametros = n_simulacoes(300, 1, 2, 5000, simulador)

clientes_restantes = parametros['clientes_restantes']


Intervalo de confiança: (0.41480921721803266, 0.5536774819125863)
Média: 0.48424334956530946



In [ ]:
#@title Inferência de ρ

trace = clientes_restantes
num_iteracoes = 12000
burn_in = 2000

rho_amostras = mcmc_rho(trace, num_iteracoes, burn_in, (-0.1, 0.1))

fig = go.Figure(go.Histogram(x=rho_amostras, name='Distribuição de ρ estimada (PDF)'))

fig.update_layout(
  height=600,
  width=800,
  title_text='Distribuição de ρ estimada (PDF)',
  xaxis_title_text='ρ',
  yaxis_title_text='PDF'
)

fig.show()

print(f'\nIntervalo de confiança: {intervalo_confianca(rho_amostras)}')
print(f'Média: {np.mean(rho_amostras)}\n')


Intervalo de confiança: (0.4515895940709179, 0.5311469449324902)
Média: 0.49136826950170404



### $m = 300, ρ = 0.1$

- $λ = 1$
- $μ = 10$

In [ ]:
parametros = n_simulacoes(300, 1, 10, 5000, simulador)

clientes_restantes = parametros['clientes_restantes']

In [ ]:
#@title Inferência de ρ

trace = clientes_restantes
num_iteracoes = 12000
burn_in = 2000

rho_amostras = mcmc_rho(trace, num_iteracoes, burn_in, (-0.1, 0.1))

fig = go.Figure(go.Histogram(x=rho_amostras, name='Distribuição de ρ estimada (PDF)'))

fig.update_layout(
  height=600,
  width=800,
  title_text='Distribuição de ρ estimada (PDF)',
  xaxis_title_text='ρ',
  yaxis_title_text='PDF'
)

fig.show()

print(f'\nIntervalo de confiança: {intervalo_confianca(rho_amostras)}')
print(f'Média: {np.mean(rho_amostras)}\n')


Intervalo de confiança: (0.05755255548635383, 0.118337552950211)
Média: 0.08794505421828241



## Casos especiais

### $m=100, y=25$

In [ ]:
#@title Inferência de ρ

trace = np.zeros(2, dtype = int)
trace[0] = 25

num_iteracoes = 12000
burn_in = 2000

rho_amostras = mcmc_rho(trace, num_iteracoes, burn_in, (-0.1, 0.1))

fig = go.Figure(go.Histogram(x=rho_amostras, name='Distribuição de ρ estimada (PDF)'))

fig.update_layout(
  height=600,
  width=800,
  title_text='Distribuição de ρ estimada (PDF)',
  xaxis_title_text='ρ',
  yaxis_title_text='PDF'
)

fig.show()

print(f'\nIntervalo de confiança: {intervalo_confianca(rho_amostras)}')
print(f'Média: {np.mean(rho_amostras)}\n')


Intervalo de confiança: (0.7837044937384261, 1.004663234724426)
Média: 0.894183864231426



### $m=100, y=100$

In [ ]:
#@title Inferência de ρ

trace = np.zeros(2, dtype = int)
trace[0] = 100

num_iteracoes = 12000
burn_in = 2000

rho_amostras = mcmc_rho(trace, num_iteracoes, burn_in, (-0.1, 0.1))

fig = go.Figure(go.Histogram(x=rho_amostras, name='Distribuição de ρ estimada (PDF)'))

fig.update_layout(
  height=600,
  width=800,
  title_text='Distribuição de ρ estimada (PDF)',
  xaxis_title_text='ρ',
  yaxis_title_text='PDF'
)

fig.show()

print(f'\nIntervalo de confiança: {intervalo_confianca(rho_amostras)}')
print(f'Média: {np.mean(rho_amostras)}\n')


Intervalo de confiança: (0.9380199816339327, 1.0031488339353731)
Média: 0.9705844077846529



### $ρ = 4/5, m = 5000$

In [ ]:
#@title Inferência do trace

rho = 4/5
m = 5000

traces = []
trace_medias = []
trace_somas = []

for i in range(300):
  trace = mcmc_trace(rho, m)

  traces.append(trace)
  trace_medias.append(np.mean(trace))
  trace_somas.append(np.sum(trace))

fig = go.Figure(go.Histogram(x=trace, name='Distribuição do trace gerado'))

fig.update_layout(
  height=600,
  width=800,
  title_text='Distribuição do trace gerado',
  xaxis_title_text='Clientes restantes no sistema',
  yaxis_title_text='PDF',
)

fig.show()

print(f'\nIntervalo de confiança: {intervalo_confianca(trace_medias)}')
print(f'Média: {np.mean(trace_medias)}')
print(f'Soma (y): {np.mean(trace_somas)}\n')


Intervalo de confiança: (2.304498164627336, 5.679693835372664)
Média: 3.9920959999999996
Soma (y): 19960.48



In [ ]:
#@title Inferência de ρ

clientes_restantes = []

for trace in traces:
  clientes_restantes.append(trace[-1])

num_iteracoes = 12000
burn_in = 2000

trace = clientes_restantes

rho_amostras = mcmc_rho(trace, num_iteracoes, burn_in, (-0.1, 0.1))

fig = go.Figure(go.Histogram(x=rho_amostras, name='Distribuição de ρ estimada (PDF)'))

fig.update_layout(
  height=600,
  width=800,
  title_text='Distribuição de ρ estimada (PDF)',
  xaxis_title_text='ρ',
  yaxis_title_text='PDF'
)

fig.show()

print(f'\nIntervalo de confiança: {intervalo_confianca(rho_amostras)}')
print(f'Média: {np.mean(rho_amostras)}\n')


Intervalo de confiança: (0.772985437886566, 0.815692585860459)
Média: 0.7943390118735125



### $ρ = 1/2, m = 5000$

In [ ]:
#@title Inferência do trace

rho = 1/2
m = 5000

traces = []
trace_medias = []
trace_somas = []

for i in range(300):
  trace = mcmc_trace(rho, m)

  traces.append(trace)
  trace_medias.append(np.mean(trace))
  trace_somas.append(np.sum(trace))

fig = go.Figure(go.Histogram(x=trace, name='Distribuição do trace gerado'))

fig.update_layout(
  height=600,
  width=800,
  title_text='Distribuição do trace gerado',
  xaxis_title_text='Clientes restantes no sistema',
  yaxis_title_text='PDF',
)

fig.show()

print(f'\nIntervalo de confiança: {intervalo_confianca(trace_medias)}')
print(f'Média: {np.mean(trace_medias)}')
print(f'Soma (y): {np.mean(trace_somas)}\n')


Intervalo de confiança: (0.8242846733743894, 1.1703206599589442)
Média: 0.9973026666666668
Soma (y): 4986.513333333333



In [ ]:
#@title Inferência de ρ

clientes_restantes = []

for trace in traces:
  clientes_restantes.append(trace[-1])

num_iteracoes = 12000
burn_in = 2000

trace = clientes_restantes

rho_amostras = mcmc_rho(trace, num_iteracoes, burn_in, (-0.1, 0.1))

fig = go.Figure(go.Histogram(x=rho_amostras, name='Distribuição de ρ estimada (PDF)'))

fig.update_layout(
  height=600,
  width=800,
  title_text='Distribuição de ρ estimada (PDF)',
  xaxis_title_text='ρ',
  yaxis_title_text='PDF'
)

fig.show()

print(f'\nIntervalo de confiança: {intervalo_confianca(rho_amostras)}')
print(f'Média: {np.mean(rho_amostras)}\n')


Intervalo de confiança: (0.45090822773666195, 0.531882758291449)
Média: 0.4913954930140555



### $ρ = 1/10, m = 5000$

In [ ]:
#@title Inferência do trace

rho = 1/10
m = 5000

traces = []
trace_medias = []
trace_somas = []

for i in range(300):
  trace = mcmc_trace(rho, m)

  traces.append(trace)
  trace_medias.append(np.mean(trace))
  trace_somas.append(np.sum(trace))

fig = go.Figure(go.Histogram(x=trace, name='Distribuição do trace gerado'))

fig.update_layout(
  height=600,
  width=800,
  title_text='Distribuição do trace gerado',
  xaxis_title_text='Clientes restantes no sistema',
  yaxis_title_text='PDF',
)

fig.show()

print(f'\nIntervalo de confiança: {intervalo_confianca(trace_medias)}')
print(f'Média: {np.mean(trace_medias)}')
print(f'Soma (y): {np.mean(trace_somas)}\n')


Intervalo de confiança: (0.09014346536836615, 0.1312098679649672)
Média: 0.11067666666666667
Soma (y): 553.3833333333333



In [ ]:
#@title Inferência de ρ

clientes_restantes = []

for trace in traces:
  clientes_restantes.append(trace[-1])

num_iteracoes = 12000
burn_in = 2000

trace = clientes_restantes

rho_amostras = mcmc_rho(trace, num_iteracoes, burn_in, (-0.1,0.1))

fig = go.Figure(go.Histogram(x=rho_amostras, name='Distribuição de ρ estimada (PDF)'))

fig.update_layout(
  height=600,
  width=800,
  title_text='Distribuição de ρ estimada (PDF)',
  xaxis_title_text='ρ',
  yaxis_title_text='PDF'
)

fig.show()

print(f'\nIntervalo de confiança: {intervalo_confianca(rho_amostras)}')
print(f'Média: {np.mean(rho_amostras)}\n')


Intervalo de confiança: (0.08425590956221987, 0.15179200259095602)
Média: 0.11802395607658794



### $ρ = 1/2, m = 100$

In [ ]:
#@title Inferência do trace

rho = 1/2
m = 100

traces = []
trace_medias = []
trace_somas = []

for i in range(300):
  trace = mcmc_trace(rho, m)

  traces.append(trace)
  trace_medias.append(np.mean(trace))
  trace_somas.append(np.sum(trace))

fig = go.Figure(go.Histogram(x=trace, name='Distribuição do trace gerado'))

fig.update_layout(
  height=600,
  width=800,
  title_text='Distribuição do trace gerado',
  xaxis_title_text='Clientes restantes no sistema',
  yaxis_title_text='PDF',
)

fig.show()

print(f'\nIntervalo de confiança: {intervalo_confianca(trace_medias)}')
print(f'Média: {np.mean(trace_medias)}')
print(f'Soma (y): {np.mean(trace_somas)}\n')


Intervalo de confiança: (-0.19821775689838605, 2.041551090231719)
Média: 0.9216666666666666
Soma (y): 92.16666666666667



In [ ]:
#@title Inferência de ρ

clientes_restantes = []

for trace in traces:
  clientes_restantes.append(trace[-1])

num_iteracoes = 12000
burn_in = 2000

trace = clientes_restantes

rho_amostras = mcmc_rho(trace, num_iteracoes, burn_in, (-0.1,0.1))

fig = go.Figure(go.Histogram(x=rho_amostras, name='Distribuição de ρ estimada (PDF)'))

fig.update_layout(
  height=600,
  width=800,
  title_text='Distribuição de ρ estimada (PDF)',
  xaxis_title_text='ρ',
  yaxis_title_text='PDF'
)

fig.show()

print(f'\nIntervalo de confiança: {intervalo_confianca(rho_amostras)}')
print(f'Média: {np.mean(rho_amostras)}\n')


Intervalo de confiança: (0.5043735368564658, 0.5799299657458975)
Média: 0.5421517513011817



## Variando o intervalo de proposta

In [ ]:
parametros = n_simulacoes(300, 1, 2, 5000, simulador)

clientes_restantes = parametros['clientes_restantes']

### Intervalo de proposta $(-0.1, 0.1)$
- $m = 300$
- $λ = 1$
- $μ = 2$

In [ ]:
#@title Inferência de ρ

trace = clientes_restantes
num_iteracoes = 12000
burn_in = 2000

rho_amostras = mcmc_rho(trace, num_iteracoes, burn_in, (-0.1,0.1))

fig = go.Figure(go.Histogram(x=rho_amostras, name='Distribuição de ρ estimada (PDF)'))

fig.update_layout(
  height=600,
  width=800,
  title_text='Distribuição de ρ estimada (PDF)',
  xaxis_title_text='ρ',
  yaxis_title_text='PDF'
)

fig.show()

print(f'\nIntervalo de confiança: {intervalo_confianca(rho_amostras)}')
print(f'Média: {np.mean(rho_amostras)}\n')


Intervalo de confiança: (0.46187642932667966, 0.5425111979779453)
Média: 0.5021938136523125



### Intervalo de proposta $(-0.5, 0.5)$
- $m = 300$
- $λ = 1$
- $μ = 2$

In [ ]:
#@title Inferência de ρ

trace = clientes_restantes
num_iteracoes = 12000
burn_in = 2000

rho_amostras = mcmc_rho(trace, num_iteracoes, burn_in, (-0.5,0.5))

fig = go.Figure(go.Histogram(x=rho_amostras, name='Distribuição de ρ estimada (PDF)'))

fig.update_layout(
  height=600,
  width=800,
  title_text='Distribuição de ρ estimada (PDF)',
  xaxis_title_text='ρ',
  yaxis_title_text='PDF'
)

fig.show()

print(f'\nIntervalo de confiança: {intervalo_confianca(rho_amostras)}')
print(f'Média: {np.mean(rho_amostras)}\n')


Intervalo de confiança: (0.4599574953732344, 0.5412957203743023)
Média: 0.5006266078737683



## Variando a priori

In [ ]:
parametros = n_simulacoes(300, 1, 2, 5000, simulador)

clientes_restantes = parametros['clientes_restantes']

### $K = 1$

In [ ]:
#@title Inferência de ρ

trace = clientes_restantes
num_iteracoes = 12000
burn_in = 2000

rho_amostras = mcmc_rho(trace, num_iteracoes, burn_in, (-0.1,0.1))

fig = go.Figure(go.Histogram(x=rho_amostras, name='Distribuição de ρ estimada (PDF)'))

fig.update_layout(
  height=600,
  width=800,
  title_text='Distribuição de ρ estimada (PDF)',
  xaxis_title_text='ρ',
  yaxis_title_text='PDF'
)

fig.show()

print(f'\nIntervalo de confiança: {intervalo_confianca(rho_amostras)}')
print(f'Média: {np.mean(rho_amostras)}\n')


Intervalo de confiança: (0.4584163727230444, 0.5358998384636535)
Média: 0.49715810559334894



### $K = 5$

In [ ]:
#@title Inferência de ρ

trace = clientes_restantes
num_iteracoes = 12000
burn_in = 2000

rho_amostras = mcmc_rho(trace, num_iteracoes, burn_in, (-0.1,0.1), K=5)

fig = go.Figure(go.Histogram(x=rho_amostras, name='Distribuição de ρ estimada (PDF)'))

fig.update_layout(
  height=600,
  width=800,
  title_text='Distribuição de ρ estimada (PDF)',
  xaxis_title_text='ρ',
  yaxis_title_text='PDF'
)

fig.show()

print(f'\nIntervalo de confiança: {intervalo_confianca(rho_amostras)}')
print(f'Média: {np.mean(rho_amostras)}\n')


Intervalo de confiança: (-0.13203509493015092, 1.1090398425303587)
Média: 0.4885023738001039

